In [1]:
import os
import shutil
import flatbuffers
import pandas as pd
from pydantic import BaseModel, Field
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Concatenate, Normalization
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from tflite_support import metadata_schema_py_generated as _metadata_fb
from tflite_support import metadata as _metadata

2025-01-13 13:57:40.758469: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-13 13:57:40.760031: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-13 13:57:40.791870: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-13 13:57:40.792436: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-13 13:57:41.290029: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
class LSTMConfig(BaseModel):
    feature_columns: list[str]
    label_names: list[str]
    label_count: int = Field(default=0)
    scaled_labels: bool
    n_steps: int = Field(default=50, ge=1)
    batch_size: int = Field(default=32, ge=1)
    epochs: int = Field(default=20, ge=1)
    test_size: float = Field(default=0.2, ge=0, le=1)
    checkpoint_path: str = Field(default="")
    loss_function: str


class TimeSeriesLSTM:

    def __init__(self, config: LSTMConfig):
        self.config = config
        self.model: Model = None

    def _create_segments(self, data: pd.Series, n_steps):
        num_samples = len(data) - n_steps + 1
        return np.array([data[i:i + n_steps] for i in range(num_samples)]).astype(np.float32)
    
    
    def dataframe_to_pred_data(self, df_prediction: pd.DataFrame):
        features = self.config.feature_columns
        n_steps = self.config.n_steps
        
        X = []
        # Create Time Series in Timestamp
        segmented_data = self._create_segments(df_prediction.index.asi8, n_steps)
        X.append(segmented_data[:, :, np.newaxis])
        
        for feature in features:
            segmented_data = self._create_segments(df_prediction[feature], n_steps)
            X.append(segmented_data[:, :, np.newaxis])
        
        return np.concatenate(X, axis=2)
    

    def dataframe_to_train_data(self, df_training: pd.DataFrame):
        features = self.config.feature_columns
        labels = self.config.label_names
        n_steps = self.config.n_steps
        
        X = self.dataframe_to_pred_data(df_training[features])
        
        label_arrays = [self._create_segments(df_training[label], n_steps)[:, -1] for label in labels]
        y = np.stack(label_arrays, axis=1)
        
        print(f"Shape of X: {X.shape}")
        print(f"Shape of y: {y.shape}")
        
        return X, y

    def load_model(self, path):
        self.model = tf.keras.models.load_model(path)
        print("Model loaded!")
        
    def save_model(self, path):
        self.model.save(path)
        print("Model saved!")

    def build_model(self, adaptation_data):
        inputs = []
        lstm_outputs = []
        
        for feature_idx in range(len(self.config.feature_columns)+1):
            if feature_idx > 0:
                feature_name = self.config.feature_columns[feature_idx-1]
            else:
                feature_name = "time"
            
            input_tensor = Input(shape=(self.config.n_steps, 1), dtype=tf.float32, name=f'input_{feature_name}')
            inputs.append(input_tensor)
            normalization = Normalization()
            normalization.adapt(adaptation_data.iloc[feature_idx])
            lstm = LSTM(64, return_sequences=False, dtype=tf.float32)
            lstm_outputs.append(lstm(normalization(input_tensor)))

        merged = Concatenate(axis=1)(lstm_outputs)
        
        x = Dense(64, activation='relu')(merged)
        if len(self.config.label_names) == 1:
            output = Dense(self.config.label_count, activation='softmax')(x)
        else:
            output = Dense(len(self.config.label_names), activation='softmax')(x)
        
        self.model = Model(inputs=inputs, outputs=output)
        self.model.compile(optimizer=Adam(), loss=self.config.loss_function, metrics=['accuracy'])
        print(self.model.summary())

    def train(self, dataframe: pd.DataFrame):
        X, y = self.dataframe_to_train_data(dataframe)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.config.test_size, random_state=69)
        
        X_train_split = np.split(X_train, indices_or_sections=3, axis=2)
        X_test_split = np.split(X_test, indices_or_sections=3, axis=2)

        X_train_squeeze = [arr.squeeze(axis=2) for arr in X_train_split]
        X_test_squeeze = [arr.squeeze(axis=2) for arr in X_test_split]

        self.model.fit(X_train_squeeze, y_train, epochs=self.config.epochs, batch_size=self.config.batch_size)
        loss = self.model.evaluate(X_test_squeeze, y_test)
        
        print(f"Loss: {loss}")

    def predict(self, input_data: pd.DataFrame):
        X = self.dataframe_to_pred_data(input_data)       
        X_split = np.split(X, indices_or_sections=3, axis=2)
        X_squeeze = [arr.squeeze(axis=2) for arr in X_split]
        return self.model.predict(X_squeeze)

    def convert_to_tflite(self, tflite_path, representative_dataset):
        print("Creating temp SavedModel")
        self.model.export("tmp")
        
        print("Converting SavedModel to tflite")
        converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir="tmp")
        converter.optimizations = {tf.lite.Optimize.DEFAULT}
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS , tf.lite.OpsSet.SELECT_TF_OPS]
        converter.inference_input_type = tf.float32
        converter.inference_output_type = tf.float32
        
        def representative_data_generator():
            X = self.dataframe_to_pred_data(representative_dataset)
            
            for i, sample in enumerate(X):
                sample = sample.reshape(1, self.config.n_steps, len(self.config.feature_columns)+1)
                if i == 0:
                    name = "time"
                else:
                    name = self.config.feature_columns[i-1]
                yield {f'input_{name}': sample}
        
        def representative_data_generatorV2():
            X = self.dataframe_to_pred_data(representative_dataset)
            feature_count = len(self.config.feature_columns)+1
            
            for i, sample in enumerate(X):
                reshaped_sample = sample.reshape(1, self.config.n_steps, feature_count)
                sample_list = np.split(reshaped_sample, indices_or_sections=feature_count, axis=2)
                yield sample_list
        
        converter.representative_dataset = lambda: representative_data_generatorV2()
        
        tflite_model = converter.convert()
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
            
        print("Removing temp SavedModel")
        shutil.rmtree("tmp")
        print("Finished!")


# Datensatz

### Es wird ein zufälliger Datensatz erzeugt

In [16]:
# Zufällige Datensätze erzeugen
train_dates = pd.date_range(start='2020-01-01', periods=10000)
pred_dates = pd.date_range(start='2021-01-01', periods=100)

train_labels = np.random.randint(low=0, high=3, size=10000)

train_data = {
    'feature1': np.random.rand(10000).astype(np.float32),
    'feature2': np.random.rand(10000).astype(np.float32),
    'feature3': np.random.rand(10000).astype(np.float32),
    'sleep': np.array([1 if lb == 0 else 0 for lb in train_labels]).astype(np.float32),
    'awake': np.array([1 if lb == 1 else 0 for lb in train_labels]).astype(np.float32),
    'rem': np.array([1 if lb == 2 else 0 for lb in train_labels]).astype(np.float32),
    'light': np.array([1 if lb == 3 else 0 for lb in train_labels]).astype(np.float32)
}

pred_data = {
    'feature1': np.random.rand(100).astype(np.float32),
    'feature2': np.random.rand(100).astype(np.float32),
    'feature3': np.random.rand(100).astype(np.float32),
}

df_train = pd.DataFrame(train_data, index=train_dates)
df_pred = pd.DataFrame(pred_data, index=pred_dates)

# Model Initialisierung

### Das Model wird mit gegebener Konfiguration initialisiert

In [3]:
# Konfiguration erstellen und Modell instanziieren
config = LSTMConfig(
    feature_columns=['time', 'movement', 'heartrate'],
    label_names=['awake','lightSleep', 'asleep',  'REM'],
    scaled_labels=False,
    n_steps=25,
    batch_size=64,
    epochs=10,
    loss_function="categorical_crossentropy"
)
ts_lstm = TimeSeriesLSTM(config=config)

2025-01-10 14:25:45.335999: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_time (InputLayer)     [(None, 25, 1)]              0         []                            
                                                                                                  
 input_movement (InputLayer  [(None, 25, 1)]              0         []                            
 )                                                                                                
                                                                                                  
 input_heartrate (InputLaye  [(None, 25, 1)]              0         []                            
 r)                                                                                               
                                                                                              

### Trainings-Step

In [26]:
# Start des Trainings
ts_lstm.train(dataframe=df_train)

AttributeError: 'LSTMConfig' object has no attribute 'label_column'

### Prediction Step

In [22]:
# Modell Predictions erzeugen

pred_results = ts_lstm.predict(df_pred)

df_pred_no_time = pd.DataFrame(pred_data)
for index, row in df_pred_no_time.iterrows():
    if index < config.n_steps:
        print("F:", row['feature1'], row['feature2'], row['feature3'], " P:", "Not enough data")
    else:
        print("F:", row['feature1'], row['feature2'], row['feature3'], " P:", pred_results[index-config.n_steps])

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
F: 0.51403713 0.19882914 0.5911012  P: Not enough data
F: 0.11934123 0.40530437 0.89096767  P: Not enough data
F: 0.38564697 0.29791084 0.19773565  P: Not enough data
F: 0.6703891 0.71410924 0.5589309  P: Not enough data
F: 0.6856324 0.26695275 0.604667  P: Not enough data
F: 0.37407333 0.5748829 0.76460654  P: Not enough data
F: 0.045838956 0.85503596 0.37089372  P: Not enough data
F: 0.11443184 0.062243324 0.710078  P: Not enough data
F: 0.35948795 0.42584494 0.7620492  P: Not enough data
F: 0.5214783 0.36130518 0.88608074  P: Not enough data
F: 0.9309409 0.4325166 0.7721853  P: Not enough data
F: 0.5279233 0.8800477 0.5392751  P: Not enough data
F: 0.33663088 0.6341414 0.3659117  P: Not enough data
F: 0.8563349 0.4537363 0.38099846  P: Not enough data
F: 0.7088956 0.10979994 0.4552031  P: Not enough data
F: 0.42051426 0.9532006 0.94734293  P: Not enough data
F: 0.62419176 0.5616601 0.22580338  P: Not enough data
F: 0.19315931 0.49989152 0.944045

# Modell Speichern, Laden und Konvertieren

In [7]:
ts_lstm.save_model("TestModel.keras")

Model saved!


In [16]:
# Lade das Modell aus der TF-Lite Datei
ts_lstm.load_model("TestModel.keras")

Model loaded!


In [26]:

class ModelSpecificInfo(BaseModel):
    name: str
    version: float
    num_inputs: int
    num_outputs: int        


_MODEL_INFO = ModelSpecificInfo(
        name="Testmodell",
        version=1.0,
        num_inputs=3,
        num_outputs=2
)


def add_metadata_to_model(model_path: str, metadata: ModelSpecificInfo):
    
    model_meta = _metadata_fb.ModelMetadataT()
    model_meta.name = metadata.name
    model_meta.description = "Ein zufälliges Modell"
    
    input_meta_0 = _metadata_fb.TensorMetadataT()
    input_meta_0.name = "Time"
    input_meta_0.dimensionNames = ['sin', 'cos']
    input_meta_0.description = "Die Tageszeit an dem der Wert aufgenommen wurde"
    input_meta_0.content = _metadata_fb.ContentT()
    input_meta_0.content.contentProperties = _metadata_fb.ValueRangeT()
    input_meta_0.content.contentProperties.min = 0
    input_meta_0.content.contentProperties.max = 10
    
    input_meta_1 = _metadata_fb.TensorMetadataT()
    input_meta_1.name = "Movement"
    input_meta_1.description = "Die Bewegungsintensität gemittelt aus den Beschleunigungssensoren"
    input_meta_1.content = _metadata_fb.ContentT()
    input_meta_1.content.contentProperties = _metadata_fb.ValueRangeT()
    input_meta_1.content.contentProperties.min = 0
    input_meta_1.content.contentProperties.max = 1
    
    input_meta_2 = _metadata_fb.TensorMetadataT()
    input_meta_2.name = "Heartrate"
    input_meta_2.description = "Die Anzahl an Herzschlägen pro Minute"
    input_meta_2.content = _metadata_fb.ContentT()
    input_meta_2.content.contentProperties = _metadata_fb.ValueRangeT()
    input_meta_2.content.contentProperties.min = 0
    input_meta_2.content.contentProperties.max = 1
    
    output_meta_1 = _metadata_fb.TensorMetadataT()
    output_meta_1.name = "Sleeplabel"
    output_meta_1.description = "Die aktuelle Schlafphase"
    

    # Creates subgraph info.
    subgraph = _metadata_fb.SubGraphMetadataT()
    subgraph.inputTensorMetadata = [input_meta_0, input_meta_1,input_meta_2]
    subgraph.outputTensorMetadata = [output_meta_1]
    model_meta.subgraphMetadata = [subgraph]
    
    b = flatbuffers.Builder(0)
    
    packed = model_meta.Pack(b)
    
    b.Finish(
        packed,
        _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER
    )
    metadata_buffer = b.Output()
    populator = _metadata.MetadataPopulator.with_model_file(model_path)
    populator.load_metadata_buffer(metadata_buffer)
    populator.populate()
    
add_metadata_to_model("TestModel.tflite", _MODEL_INFO)

ValueError: The number of input tensors (1) should match the number of input tensor metadata (3)

In [5]:
df_train = pd.read_json("data/4_squashed_format/2024-08-21.json", dtype=np.float32)
df_train = df_train.dropna()

/tmp/ipykernel_68465/2381353175.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_train = pd.read_json("data/4_squashed_format/2024-08-21.json", dtype=np.float32)


### Sparse Categorical Crossentropy

In [3]:
feature_columns = [
    "movementData",
    #"sleepLevels",
    #"restlessMoments",
    "heartrate",
    #"stress",
    #"bodyBattery",
    #"hrv"
]

label_names = ["sleepLevels"]

config = LSTMConfig(
    feature_columns=feature_columns,
    label_names=label_names,
    label_count=4,
    n_steps=120,
    batch_size=32,
    epochs=2,
    scaled_labels=True,
    checkpoint_path="models/chkp.keras",
    loss_function="sparse_categorical_crossentropy"
)
ts_lstm = TimeSeriesLSTM(config=config)

### Categorical Crossentropy

In [32]:
df_train['awake'] = [1 if d == 0 else 0 for d in df_train['sleepLevels']]
df_train['asleep'] = [1 if d == 1 else 0 for d in df_train['sleepLevels']]
df_train['lightSleep'] = [1 if d == 2 else 0 for d in df_train['sleepLevels']]
df_train['rem'] = [1 if d == 3 else 0 for d in df_train['sleepLevels']]

feature_columns = [
    "movementData",
    #"sleepLevels",
    #"restlessMoments",
    "heartrate",
    #"stress",
    #"bodyBattery",
    #"hrv"
]

label_names = ["awake", "lightSleep", "asleep", "rem"]

config = LSTMConfig(
    feature_columns=feature_columns,
    label_names=label_names,
    n_steps=120,
    batch_size=32,
    epochs=2,
    scaled_labels=True,
    checkpoint_path="models/chkp.keras",
    loss_function="categorical_crossentropy"
)
ts_lstm = TimeSeriesLSTM(config=config)

### Model Initialisation

In [6]:
ts_lstm.build_model(df_train[feature_columns])
ts_lstm.train(dataframe=df_train[feature_columns + label_names])
print("Initial Training Completed!")

ts_lstm.convert_to_tflite("models/sleepPhaseAnalyserV1.tflite", df_train)

2025-01-13 13:58:50.463254: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-01-13 13:58:50.463636: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_time (InputLayer)     [(None, 120, 1)]             0         []                            
                                                                                                  
 input_movementData (InputL  [(None, 120, 1)]             0         []                            
 ayer)                                                                                            
                                                                                                  
 input_heartrate (InputLaye  [(None, 120, 1)]             0         []                            
 r)                                                                                               
                                                                                              

INFO:tensorflow:Assets written to: tmp/assets


Saved artifact at 'tmp'. The following endpoints are available:

* Endpoint 'serve'
  Args:
    args_0: [<1>, <2>, <3>]
      <1>: float32 Tensor, shape=(None, 120, 1)
      <2>: float32 Tensor, shape=(None, 120, 1)
      <3>: float32 Tensor, shape=(None, 120, 1)
  Returns:
    float32 Tensor, shape=(None, 4)
Converting SavedModel to tflite


2025-01-13 13:59:41.631150: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-01-13 13:59:41.631172: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-01-13 13:59:41.631455: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: tmp
2025-01-13 13:59:41.633815: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-01-13 13:59:41.633831: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: tmp
2025-01-13 13:59:41.638818: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-01-13 13:59:41.640213: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-01-13 13:59:41.706766: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: tmp
2025-01-13 13:59:41.748571: I tensorflow/cc/sa

Removing temp SavedModel
Finished!


fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32


In [91]:
ts_lstm.load_model("models/sleepPhaseAnalyserV1.keras")

Model loaded!


In [39]:
ts_lstm.save_model("models/sleepPhaseAnalyserV1.keras")

Model saved!


In [49]:
ts_lstm.convert_to_tflite("models/sleepPhaseAnalyserV1.tflite", df_train)

Creating temp SavedModel
INFO:tensorflow:Assets written to: tmp/assets


INFO:tensorflow:Assets written to: tmp/assets


Saved artifact at 'tmp'. The following endpoints are available:

* Endpoint 'serve'
  Args:
    args_0: [<1>, <2>, <3>]
      <1>: float32 Tensor, shape=(None, 120, 1)
      <2>: float32 Tensor, shape=(None, 120, 1)
      <3>: float32 Tensor, shape=(None, 120, 1)
  Returns:
    float32 Tensor, shape=(None, 4)
Converting SavedModel to tflite


2025-01-13 12:37:28.019413: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-01-13 12:37:28.019443: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-01-13 12:37:28.019587: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: tmp
2025-01-13 12:37:28.021147: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-01-13 12:37:28.021160: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: tmp
2025-01-13 12:37:28.025278: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-01-13 12:37:28.081988: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: tmp
2025-01-13 12:37:28.120295: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 100707 microseconds.
2025-01-13 12:37:28.22077

ValueError: cannot reshape array of size 360 into shape (1,120,2)

# Utils

In [93]:
def list_all_files(path):
    try:
        files = []

        for f in os.listdir(path):
            filepath = os.path.join(path, f)
            if os.path.isfile(filepath):  # Prüft, ob es sich um eine Datei handelt
                files.append(f)

        return files
    except FileNotFoundError:
        print(f"Das Verzeichnis '{path}' wurde nicht gefunden.")
        return []

In [95]:


ts_lstm.load_model("models/sleepPhaseAnalyserV1.keras")

files = list_all_files("data/4_squashed_format/")

for file in files:
    print("Start Training with " + file)
    df_train = pd.read_json("data/4_squashed_format/" + file, dtype=np.float32)
    df_train = df_train.dropna()
    
    ts_lstm.train(dataframe=df_train[feature_columns + label_names])
    ts_lstm.save_model("models/sleepPhaseAnalyserV1.keras")
print("Training completed!")

Start Training with2024-08-23.json


/tmp/ipykernel_97970/1478985355.py:5: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_train = pd.read_json("data/4_squashed_format/" + file, dtype=np.float32)


Epoch 1/2
704/704 [==============================] - 23s 29ms/step - loss: 0.2386 - accuracy: 0.7857
Epoch 2/2
176/176 [==============================] - 3s 9ms/step - loss: 0.2405 - accuracy: 0.7806
Loss: [0.2404816746711731, 0.7806004881858826]
Model saved!
Start Training with2024-09-05.json


/tmp/ipykernel_97970/1478985355.py:5: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_train = pd.read_json("data/4_squashed_format/" + file, dtype=np.float32)


Epoch 1/2
763/763 [==============================] - 23s 30ms/step - loss: 0.1577 - accuracy: 0.8578
Epoch 2/2
191/191 [==============================] - 2s 9ms/step - loss: 0.1431 - accuracy: 0.8609
Loss: [0.14311617612838745, 0.8608880639076233]
Model saved!
Start Training with2024-09-08.json


...

Epoch 1/2
644/644 [==============================] - 19s 29ms/step - loss: 0.2676 - accuracy: 0.7725
Epoch 2/2
161/161 [==============================] - 2s 11ms/step - loss: 0.2615 - accuracy: 0.7789
Loss: [0.2615232765674591, 0.7788573503494263]
Model saved!
Start Training with2024-08-30.json


/tmp/ipykernel_97970/1478985355.py:5: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_train = pd.read_json("data/4_squashed_format/" + file, dtype=np.float32)


Epoch 1/2
747/747 [==============================] - 25s 34ms/step - loss: 0.2054 - accuracy: 0.8287
Epoch 2/2
187/187 [==============================] - 2s 9ms/step - loss: 0.1922 - accuracy: 0.8417
Loss: [0.19219979643821716, 0.8417266011238098]
Model saved!
Start Training with2024-09-02.json


/tmp/ipykernel_97970/1478985355.py:5: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_train = pd.read_json("data/4_squashed_format/" + file, dtype=np.float32)


Epoch 1/2
693/693 [==============================] - 21s 30ms/step - loss: 0.3681 - accuracy: 0.7315
Epoch 2/2
174/174 [==============================] - 2s 11ms/step - loss: 0.3597 - accuracy: 0.7287
Loss: [0.3596736788749695, 0.7286514043807983]
Model saved!
Training completed!
